Cleaner version of the notebook that I run on laptop to scape data faster

In [60]:
#import libraries
from time import time, sleep
import random
import urllib
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
import bs4
from bs4 import BeautifulSoup
import re
from IPython.display import clear_output
import os

In [61]:
#captcha alert settings
import winsound
duration = 5000  # milliseconds
freq = 440  # Hz

In [62]:
file = 'jobs1.csv'

In [63]:
try: 
    df_more = pd.read_csv(file , sep='\t', encoding='utf-8', index_col=0)
    print('file successfully opened')
except:
    print('file not found, creating new file')
    df_more = pd.DataFrame(columns=["Title","Location","Company","Salary", "Synopsis"])
    df_more.to_csv(file, sep='\t', encoding='utf-8')

file successfully opened


In [64]:
df_more

,Title,Location,Company,Salary,Synopsis
0,Junior Data Scientist,"Austin, TX 78735",Kestra Financial,$84.2K - $107K a year,"Create complex data pipelines, while learning ..."
1,Data Scientist I (22-0623),"Austin, TX 78741 (Pleasant Valley area)",Office of the Attorney General of Texas,None,"Develops data quality measures, analyzes data ..."
2,Data Scientist,"Austin, TX 78701 (Downtown area)",Zilliant,$110K - $140K a year,Experience implementing data science routines ...
3,Data Scientist,"Hybrid remote in Austin, TX",YETI,$103K - $131K a year,Experience with data visualization tools (e.g....
4,Junior Data Scientist,"Austin, TX",ANALYTOS,$67.6K - $85.6K a year,Designs and drives the creation of new standar...
...,...,...,...,...,...
55246,Senior Business Data Analyst,"Fort Lauderdale, FL",Momentum Consulting,$99.4K - $126K a year,Provide data analysis and interpretation.Produ...
55247,Senior Machine Learning Engineer,"Remote in Miami, FL",Wikimedia Foundation,$111K - $141K a year,This includes communicating with the customers...
55248,Principal ML & NLP Engineer,"Remote in Miami, FL",Recruiting From Scratch,None,Given you are constructing the foundation on w...
55249,"Sr. Manager, Data Science and Governance, Popeyes","Miami, FL",Restaurant Brands International,$49.3K - $62.5K a year,Familiarity with data engineering (liaison lev...


In [65]:
headers_list = [
# Firefox 77 Mac
{
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
"Accept-Language": "en-US,en;q=0.5",
"Referer": "https://www.google.com/",
"DNT": "1",
"Connection": "keep-alive",
"Upgrade-Insecure-Requests": "1"
},
# Firefox 77 Windows
{
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
"Accept-Language": "en-US,en;q=0.5",
"Accept-Encoding": "gzip, deflate, br",
"Referer": "https://www.google.com/",
"DNT": "1",
"Connection": "keep-alive",
"Upgrade-Insecure-Requests": "1"
},
# Chrome 83 Mac
{
"Connection": "keep-alive",
"DNT": "1",
"Upgrade-Insecure-Requests": "1",
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"Sec-Fetch-Site": "none",
"Sec-Fetch-Mode": "navigate",
"Sec-Fetch-Dest": "document",
"Referer": "https://www.google.com/",
"Accept-Encoding": "gzip, deflate, br",
"Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
},
# Chrome 83 Windows 
{
"Connection": "keep-alive",
"Upgrade-Insecure-Requests": "1",
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"Sec-Fetch-Site": "same-origin",
"Sec-Fetch-Mode": "navigate",
"Sec-Fetch-User": "?1",
"Sec-Fetch-Dest": "document",
"Referer": "https://www.google.com/",
"Accept-Encoding": "gzip, deflate, br",
"Accept-Language": "en-US,en;q=0.9"
}
]

In [66]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 5000
cities = ['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
            'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
            'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami']
cities1 = ['Boston', 'Charlottesville', 'Richmond', 'Baltimore', 'Harrisonburg', 'San+Antonio', 'San+Diego', 'San+Jose',
                 'Austin', 'Jacksonville', 'Indianapolis', 'Columbus', 'Fort+Worth', 'Charlotte', 'Detroit', 'El+Paso', 
                 'Memphis', 'Orlando', 'Nashville', 'Louisville', 'Milwaukee', 'Las+Vegas', 'Albuquerque', 'Tucson' ]

In [76]:
i = 0
count = 0
results = []

for city in set(cities1):
    for start in range(0, max_results_per_city, 10):
        # Grab the results from the request (as above)
        # Append to the full set of results
        url = url_template.format(city, start)
        headers = random.choice(headers_list)
        r = requests.Session()
        r.headers = headers
        html = r.get(url)
        #html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
        #captcha detector
        if 'hCaptcha solve page' in soup.get_text():
            print('captcha detected')
            winsound.Beep(freq, duration) #sound alert, might get irritating, probably won't work on mac
            input("Press Enter to continue...")#this would pause the code until i press enter
        for each in soup.find_all(class_= "result" ):
            try: 
                title = each.find(class_='jobTitle').text.replace('\n', '').replace('new', '')
            except:
                title = 'None'
            try:
                location = each.find('div', {'class':"companyLocation" }).text.replace('\n', '')
            except:
                location = 'None'
            try: 
                company = each.find(class_='companyName').text.replace('\n', '')
            except:
                company = 'None'
            try:
                salary = each.find('span', {'class':'estimated-salary'}).text.replace('\n', '').replace('Estimated', '')
            except:
                salary = 'None'
            synopsis = each.find('div', {'class':'job-snippet'}).text.replace('\n', '')
            df_more = df_more.append({'Title':title, 'Location':location, 'Company':company, 'Salary':salary, 'Synopsis':synopsis}, ignore_index=True)
            i += 1
            if i % 1000 == 0:
                print('You have ' + str(i) + ' results. ' + str(df_more.dropna().drop_duplicates().shape[0]) + " of these aren't rubbish.")
        #count the number of pages scraped and print, so I know how far the code is running
        count+= 1
        print(count, 'loops done, proceed to sleep, current city is ', city)
        df_more.to_csv(file, sep='\t', encoding='utf-8')#write every page so i don't lose my data again
        sleep(random.randint(7,20))
        #I don't want too see long line of the process, clear_output will clear previous lines
        clear_output(wait=True)
        print(df_more.shape, 'data saved, last visited city is ', city, ', continuing....')
    print('moving to next city')
    sleep(random.randint(30,60))

(72226, 5) data saved, last visited city is  Harrisonburg , continuing....
1328 loops done, proceed to sleep, current city is  Harrisonburg


KeyboardInterrupt: 

In [68]:
print(df_more.duplicated().sum())

53461


In [69]:
df_more[~df_more.duplicated()]

,Title,Location,Company,Salary,Synopsis
0,Junior Data Scientist,"Austin, TX 78735",Kestra Financial,$84.2K - $107K a year,"Create complex data pipelines, while learning ..."
1,Data Scientist I (22-0623),"Austin, TX 78741 (Pleasant Valley area)",Office of the Attorney General of Texas,None,"Develops data quality measures, analyzes data ..."
2,Data Scientist,"Austin, TX 78701 (Downtown area)",Zilliant,$110K - $140K a year,Experience implementing data science routines ...
3,Data Scientist,"Hybrid remote in Austin, TX",YETI,$103K - $131K a year,Experience with data visualization tools (e.g....
4,Junior Data Scientist,"Austin, TX",ANALYTOS,$67.6K - $85.6K a year,Designs and drives the creation of new standar...
...,...,...,...,...,...
55271,GCP Sr. Big Data Engineer,"Irving, TX",Kairos Technologies,None,Experience in data visualization tools like Ki...
55272,Senior Marketing Data Analyst,"Remote in Arlington, TX",Circle,$87.6K - $111K a year,Identifying trends in the data to inform ongoi...
55273,AI Developer,"Southlake, TX 76092",Avodah,$103K - $130K a year,Design data acquisition and data labeling syst...
55274,Principal ML & NLP Engineer,"Remote in Watauga, TX 76148+1 location",Recruiting From Scratch,None,Given you are constructing the foundation on w...


In [47]:
df = pd.read_csv('jobs.csv' , sep='\t', encoding='utf-8', index_col=0)

In [48]:
df[~df.duplicated()]

,Title,Location,Company,Salary,Synopsis
0,Data Scientist,"Remote in Charlottesville, VA 22902+2 locations",S&P Global,None,3+ years of experience in data analysis or sim...
1,9070 - Intelligence Data Scientist*,"Charlottesville, VA 22911",Integral,$101K - $128K a year,Should be skilled in data visualization and us...
2,Bioinformatics Data Scientist,"Charlottesville, VA 22911","Signature Science, LLC",None,A bioinformatics data scientist is responsible...
3,Data Scientist - Mid,"Charlottesville, VA+1 location",Celestar Corporation,$115K - $146K a year,Demonstrates knowledge of data transfer requir...
4,Data Scientist (Multiple Locations) - Top Secr...,"Charlottesville, VA",Logistics Management Institute,$127K - $161K a year,Advanced data science experience required.Demo...
...,...,...,...,...,...
84439,Technical Support Specialist,"Rockville, MD 20850",GeneCopoeia,$39.8K - $50.4K a year,Help with the interpretation and analysis of c...
84440,Senior Communications Specialist,"Arlington, VA","Eastern Research Group, Inc.",$70.8K - $89.6K a year,Collaborating across multidisciplinary teams i...
84441,Bioinformatics Research Associate I - Gynecolo...,"Annandale, VA",Henry M Jackson Foundation For The Advan,None,Integrate bioinformatics data analysis with pu...
84668,Receptionist,"Washington, DC 20005 (Downtown area)","Natural Resources Defense Council, Inc.",None,Maintain and update various data sets and prov...


In [70]:
frames = [df, df_more]

In [71]:
result = pd.concat(frames)

In [72]:
result

,Title,Location,Company,Salary,Synopsis
0,Data Scientist,"Remote in Charlottesville, VA 22902+2 locations",S&P Global,None,3+ years of experience in data analysis or sim...
1,9070 - Intelligence Data Scientist*,"Charlottesville, VA 22911",Integral,$101K - $128K a year,Should be skilled in data visualization and us...
2,Bioinformatics Data Scientist,"Charlottesville, VA 22911","Signature Science, LLC",None,A bioinformatics data scientist is responsible...
3,Data Scientist - Mid,"Charlottesville, VA+1 location",Celestar Corporation,$115K - $146K a year,Demonstrates knowledge of data transfer requir...
4,Data Scientist (Multiple Locations) - Top Secr...,"Charlottesville, VA",Logistics Management Institute,$127K - $161K a year,Advanced data science experience required.Demo...
...,...,...,...,...,...
55286,NLP Engineer at Speech to Text Start Up,"Remote in Watauga, TX 76148+1 location",Recruiting From Scratch,$123K - $155K a year,Their AI-powered collaborative note-taking app...
55287,GCP Sr. Big Data Engineer,"Irving, TX",Kairos Technologies,None,Experience in data visualization tools like Ki...
55288,Senior Data Scientist - Marketing Analytics,"Hybrid remote in Irving, TX 75038",Cottonwood Financial,None,Understanding of credit bureau data and credit...
55289,Principal ML & NLP Engineer,"Remote in Watauga, TX 76148+1 location",Recruiting From Scratch,None,Given you are constructing the foundation on w...


In [73]:
result = result[~result.duplicated()].reset_index(drop=True)

In [74]:
result

,Title,Location,Company,Salary,Synopsis
0,Data Scientist,"Remote in Charlottesville, VA 22902+2 locations",S&P Global,None,3+ years of experience in data analysis or sim...
1,9070 - Intelligence Data Scientist*,"Charlottesville, VA 22911",Integral,$101K - $128K a year,Should be skilled in data visualization and us...
2,Bioinformatics Data Scientist,"Charlottesville, VA 22911","Signature Science, LLC",None,A bioinformatics data scientist is responsible...
3,Data Scientist - Mid,"Charlottesville, VA+1 location",Celestar Corporation,$115K - $146K a year,Demonstrates knowledge of data transfer requir...
4,Data Scientist (Multiple Locations) - Top Secr...,"Charlottesville, VA",Logistics Management Institute,$127K - $161K a year,Advanced data science experience required.Demo...
...,...,...,...,...,...
3800,Senior Data Scientist/Machine Learning Engineer,"Remote in Watauga, TX 76148+14 locations",Recruiting From Scratch,None,"Work with cross-functional teams, including la..."
3801,NLP Engineer at Speech to Text Start Up,"Remote in Watauga, TX 76148+1 location",Recruiting From Scratch,$123K - $155K a year,Their AI-powered collaborative note-taking app...
3802,Data Plane Engineer,"Remote in Fort Worth, TX+2 locations",Recruiting From Scratch,None,Troubleshooting and optimizing data path.Archi...
3803,Senior Marketing Data Analyst,"Remote in Arlington, TX",Circle,$87.6K - $111K a year,Identifying trends in the data to inform ongoi...


In [75]:
result.to_csv('data.csv', sep='\t', encoding='utf-8')